# Generate W2V

## Data download

### big data

In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/beeradvocate.json.gz
#!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/googlelocal/reviews.clean.json.gz

!gzip -d /content/beeradvocate.json.gz
#!gzip -d /content/reviews.clean.json.gz

f = open('/content/beeradvocate.json')
#f = open('/content/reviews.clean.json')

import re
import json
st=f.read()
data = st.replace("\"","'")
data = data.replace("\'","'")
data = data.replace("{'", "{\"")
data = data.replace("': '", "\": \"")
data = data.replace("', '", "\", \"")
data = data.replace("'}", "\"}").replace("\\'","'").replace("\\t"," ")
data = re.sub(r'[^\x00-\x7F]',' ', data)
#data = data.replace("': u'", "\": \"")
#data = data.replace("'], '", "\"], \"")
#data = data.replace("': [u'", "\": [u\"")

lst=data.split("\n")
f.close()

import gc
gc.collect()


--2023-05-26 13:46:43--  https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/beeradvocate.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453029619 (432M) [application/x-gzip]
Saving to: ‘beeradvocate.json.gz’

beeradvocate.json.g 100%[===================>] 432.04M  39.7MB/s    in 8.9s    

2023-05-26 13:46:53 (48.4 MB/s) - ‘beeradvocate.json.gz’ saved [453029619/453029619]



In [ ]:
!pip install unidecode
from unidecode import unidecode



fil=open("data.txt","w")
for i in range(len(lst)):
  #print(lst[i])
  try:
    s=eval(lst[i])['review/text']
    fil.write(s+"\n")
  except:
    print(lst[i])
    continue
fil.close()
lst.clear()

gc.collect()



'''
#print(lst[0])
fil=open("data.txt","w")
for i in range(len(lst)):
  #print(lst[i])
  try:
    s=eval(lst[i])['reviewText']
    s=unidecode(s)
    fil.write(s+"\n")
  except:
    continue
fil.close()
lst.clear()

gc.collect()
'''

### gutenberg data

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('gutenberg')
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.     import Word2Vec as w2v

### small data

In [ ]:
import nltk
nltk.download('gutenberg')
!cp /root/nltk_data/corpora/gutenberg/austen-emma.txt sample.txt


### train w2v

In [ ]:
gc.collect()

sw = stopwords.words('english')
# Function to remove stopwords per line if they are present in the line
def rem_stops_line(line, words):
    if len(line) >1:
        return [w for w in line if w not in words]
    else:
        return line

# Remove stop words for an entire text. Separate functions make it easier to parallelize if required.
def remove_stops(text, words = sw):
    return [rem_stops_line(line, words) for line in text]

# Open the file and convert it to a list of lines
with open('data.txt', 'r') as f:  #sample.txt
    lines = f.readlines()

# Remove new lines, convert all to lowercase, remove punctuation and stop words and tokenise
lines = [line.rstrip('\n').lower() for line in lines]
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]
lines = [word_tokenize(line) for line in lines]
filtered_lines = remove_stops(text = lines, words = sw)

print(filtered_lines[:10])

lines.clear()
gc.collect()

### Training using the setup

In [ ]:
w = w2v(
    filtered_lines,
    min_count=3,
    sg = 1,       # 1 for skip-gram, 0 for cbow
    window=7   # sliding window size
)


### Save word2vec

In [ ]:
w.wv.save_word2vec_format("generated_w2v.txt") # save to disk

### find similar word based on the training

In [ ]:
print(w.wv.most_similar('clove'))


In [ ]:
!python -m gensim.scripts.word2vec2tensor -i "generated_w2v.txt" -o "model"


#Extra

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk

scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')
article = scrapped_data .read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text



In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format("wvecemma", binary=True) # load from disk


FileNotFoundError: ignored

In [ ]:
article_text

'\nArtificial intelligence (AI) is intelligence—perceiving, synthesizing, and inferring information—demonstrated by machines, as opposed to intelligence displayed by humans or by other animals. Example tasks in which this is done include speech recognition, computer vision, translation between (natural) languages, as well as other mappings of inputs.\nAI applications include advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), understanding human speech (such as Siri and Alexa), self-driving cars (e.g., Waymo), generative or creative tools (ChatGPT and AI art), automated decision-making, and competing at the highest level in strategic game systems (such as chess and Go).[1]\nAs machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.[2] For instance, optical character recognition is frequently excluded from things conside

In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/beeradvocate.json.gz

--2023-05-19 08:30:49--  https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/beeradvocate.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453029619 (432M) [application/x-gzip]
Saving to: ‘beeradvocate.json.gz’

beeradvocate.json.g 100%[===================>] 432.04M  15.4MB/s    in 30s     

2023-05-19 08:31:20 (14.2 MB/s) - ‘beeradvocate.json.gz’ saved [453029619/453029619]



In [ ]:
!gzip -d /content/beeradvocate.json.gz

In [ ]:
import pandas as pd
df = pd.read_json('/content/beeradvocate.json', orient='split')
df.info()

#import json
  
#f = open('/content/beeradvocate.json')

'''
data = json.load(f)

print(data)  
for i in data['emp_details']:
    print(i)
  
f.close()
'''

ValueError: ignored

In [ ]:
ddata = json.load(f)

TypeError: ignored

In [ ]:
f = open('/content/beeradvocate.json')

In [ ]:
import json
st=f.read()
#print(st[:100])
#st="{"+st+"}"
#print(st[:100])
data = st.replace("\"","'")
data = data.replace("\'","'")
data = data.replace("{'", "{\"")
data = data.replace("': '", "\": \"")
data = data.replace("', '", "\", \"")
data = data.replace("'}", "\"}").replace("\\'","'").replace("\\t"," ")
lst=data.split("\n")
#print(lst[1]['review/text'])
#data = json.loads(data)

In [ ]:
print(type(lst[10]))
#print(eval(str(lst[10]))['review/text'])

<class 'str'>


In [ ]:
fil=open("data.txt","w")
for i in range(len(lst)):
  s=eval(lst[0])['review/text']
  fil.write(s+"\n")
fil.close()

In [ ]:
'''
fil=open("data.txt","w")
for i in range(len(lst)):
  s=eval(lst[i])['review/text']
  fil.write(s+"\n")
fil.close()
'''

'\nfil=open("data.txt","w")\nfor i in range(len(lst)):\n  s=eval(lst[i])[\'review/text\']\n  fil.write(s+"\n")\nfil.close()\n'

In [ ]:
#!wget https://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove.6B.zip


import numpy as np

# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
glove_file = datapath('/content/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

<ipython-input-11-a15a8593b3ca>:19: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


In [ ]:
def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

In [ ]:
display_pca_scatterplot(model, 
                        ['coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
                         'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
                         'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
                         'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
                         'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
                         'homework', 'assignment', 'problem', 'exam', 'test', 'class',
                         'school', 'college', 'university', 'institute'])

<IPython.core.display.Javascript object>

In [ ]:
display_pca_scatterplot(model, sample=300)

AttributeError: ignored

In [ ]:
#!unzip "/content/drive/MyDrive/all_data/train.csv.zip"
import nltk
nltk.download('all')

import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('train.csv').sample(50000, random_state=23)


STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

def clean_dataframe(data):
    "drop nans, then apply 'clean_sentence' function to question1 and 2"
    data = data.dropna(how="any")
    
    for col in ['question1', 'question2']:
        data[col] = data[col].apply(clean_sentence)
    
    return data

data = clean_dataframe(data)
data.head(5)

def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for col in ['question1', 'question2']:
        for sentence in data[col].iteritems():
            word_list = sentence[1].split(" ")
            corpus.append(word_list)
            
    return corpus

corpus = build_corpus(data)        
corpus[0:2]

model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=200, workers=4)
model.wv['trump']

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

TypeError: ignored

In [ ]:
# A more selective model
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=500, workers=4)
tsne_plot(model)


In [ ]:
# A less selective model
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=100, workers=4)
tsne_plot(model)